In [32]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
import torchvision

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore')

In [33]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(1) # Seed 고정

In [39]:
df = pd.read_csv("train.csv")

In [40]:
df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3
2694,TRAIN_2694,./train/TRAIN_2694.mp4,5
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0


In [36]:
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list):
        self.video_path_list = video_path_list
        self.label_list = label_list
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(50):
            _, img = cap.read()
            img = cv2.resize(img, (128, 128))
            img = img / 255.
            frames.append(img)
        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)

In [37]:
train_dataset = CustomDataset(train['video_path'].values, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size = 8, shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['video_path'].values, val['label'].values)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle=False, num_workers=0)

In [38]:
train["label"].values.shape

(2158,)

In [7]:
train_dataset[0]

(tensor([[[[0.9882, 0.9882, 0.9882,  ..., 0.9922, 0.9922, 0.9922],
           [0.9882, 0.9882, 0.9882,  ..., 0.9922, 0.9922, 0.9922],
           [0.9882, 0.9882, 0.9882,  ..., 0.9922, 0.9922, 0.9922],
           ...,
           [0.1725, 0.1804, 0.1961,  ..., 0.1137, 0.1059, 0.0941],
           [0.2353, 0.2078, 0.1961,  ..., 0.1137, 0.1059, 0.0941],
           [0.3647, 0.3216, 0.2824,  ..., 0.1137, 0.1059, 0.0941]],
 
          [[0.9882, 0.9882, 0.9882,  ..., 0.9882, 0.9922, 0.9922],
           [0.9882, 0.9882, 0.9882,  ..., 0.9882, 0.9882, 0.9882],
           [0.9882, 0.9882, 0.9882,  ..., 0.9882, 0.9922, 0.9922],
           ...,
           [0.3569, 0.4078, 0.4078,  ..., 0.0667, 0.0863, 0.0824],
           [0.2627, 0.3294, 0.3882,  ..., 0.0902, 0.0863, 0.0824],
           [0.1804, 0.2118, 0.3020,  ..., 0.1294, 0.1137, 0.1020]],
 
          [[0.9882, 0.9882, 0.9882,  ..., 0.9882, 0.9882, 0.9882],
           [0.9882, 0.9882, 0.9882,  ..., 0.9882, 0.9882, 0.9882],
           [0.9922, 0.99

In [8]:
train_dataset[0][0].shape

torch.Size([3, 50, 128, 128])

In [9]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
device

device(type='cuda')

In [10]:
def conv3x3x3(in_channels, out_channels, stride = 1):
    return nn.Conv3d(in_channels,
                    out_channels,
                    kernel_size = 3,
                    stride = stride,
                    padding = 1,
                    bias = False)

def conv1x1x1(in_channels, out_channels, stride= 1):
    return nn.Conv3d(in_channels,
                    out_channels,
                    kernel_size = 1,
                    stride = stride,
                    bias = False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super().__init__()

        self.conv1 = conv3x3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU(inplace = True)

        # in_channels -> out_channels 

        self.conv2 = conv3x3x3(out_channels, out_channels) # channel 유지
        self.bn2 = nn.BatchNorm3d(out_channels)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class BottleNeck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super().__init__()

        self.conv1 = conv1x1x1(in_channels, out_channels)
        self.bn1 = nn.BatchNorm3d(out_channels)
        self.conv2 = conv3x3x3(out_channels, out_channels, stride)
        self.bn2 = nn.BatchNorm3d(out_channels)
        self.conv3 = conv1x1x1(out_channels, out_channels * self.expansion)
        self.bn3 = nn.BatchNorm3d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace = True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x) 
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual 
        out = self.relu(out)
        
        return out

In [11]:
x = next(iter(train_loader))[0]
print(x.size())
conv1 = nn.Conv3d(3, 64, (7, 7, 7), (1, 2, 2), padding = (3, 3, 3), bias = False)
x = conv1(x)
print(x.size())
maxpool = nn.MaxPool3d(kernel_size = 3, stride = 2, padding = 1)
x = maxpool(x)
print(x.size())

torch.Size([8, 3, 50, 128, 128])
torch.Size([8, 64, 50, 64, 64])
torch.Size([8, 64, 25, 32, 32])


In [12]:
class ResNet50(nn.Module):
    def __init__(self, block = BottleNeck, layers = [3, 4, 6, 3], block_inplanes = [64, 128, 256, 512], input_channels = 3, conv1_t_size = 7, conv1_t_stride = 1, no_max_pool = False, n_classes = 13):
        super(ResNet50, self).__init__()

        self.in_channels = block_inplanes[0]
        self.no_max_pool = no_max_pool

        self.conv1 = nn.Conv3d(input_channels,
                                self.in_channels,
                                kernel_size = (conv1_t_size, 7, 7),
                                stride = (conv1_t_stride, 2, 2),
                                padding = (conv1_t_size // 2, 3, 3),
                                bias = False)

        self.bn1 = nn.BatchNorm3d(self.in_channels)
        self.relu = nn.ReLU(inplace = True)
        self.maxpool = nn.MaxPool3d(kernel_size = 3, stride = 2, padding = 1)
        self.layer1 = self.make_layer(block, block_inplanes[0], layers[0])
        self.layer2 = self.make_layer(block, block_inplanes[1], layers[1], stride = 2)
        self.layer3 = self.make_layer(block, block_inplanes[2], layers[2], stride = 2)
        self.layer4 = self.make_layer(block, block_inplanes[3], layers[3], stride = 2)
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.classifier = nn.Linear(block_inplanes[3] * block.expansion, n_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight,
                                        mode = "fan_out",
                                        nonlinearity = "relu")
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def make_layer(self, block, out_channels, blocks, stride = 1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                                        conv1x1x1(self.in_channels, out_channels * block.expansion, stride),
                                        nn.BatchNorm3d(out_channels * block.expansion)
                                        )
        layers = []
        layers.append(
            block(in_channels = self.in_channels,
                out_channels = out_channels,
                stride = stride,
                downsample = downsample))

        self.in_channels = out_channels * block.expansion
        
        
        for i in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        batch_size = x.size(0)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if not self.no_max_pool:
            x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(batch_size, -1)
        x = self.classifier(x)
        
        return x

In [13]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, 50):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [14]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score
    

In [15]:
model = ResNet50()
model = nn.DataParallel(model, device_ids = [0, 1])

model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.40570] Val Loss : [1.35299] Val F1 : [0.08014]


  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

...

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [48], Train Loss : [0.06611] Val Loss : [0.81862] Val F1 : [0.30095]
Epoch 00048: reducing learning rate of group 0 to 4.8828e-07.


  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [49], Train Loss : [0.06880] Val Loss : [0.82518] Val F1 : [0.32073]


In [16]:
test = pd.read_csv("test.csv")
test

,sample_id,video_path
0,TEST_0000,./test/TEST_0000.mp4
1,TEST_0001,./test/TEST_0001.mp4
2,TEST_0002,./test/TEST_0002.mp4
3,TEST_0003,./test/TEST_0003.mp4
4,TEST_0004,./test/TEST_0004.mp4
...,...,...
1795,TEST_1795,./test/TEST_1795.mp4
1796,TEST_1796,./test/TEST_1796.mp4
1797,TEST_1797,./test/TEST_1797.mp4
1798,TEST_1798,./test/TEST_1798.mp4


In [18]:
test_dataset = CustomDataset(test['video_path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = 8, shuffle=False, num_workers=0)

In [19]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [20]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/225 [00:00<?, ?it/s]

In [26]:
submit = pd.read_csv('sample_submission.csv')

In [27]:
submit

,sample_id,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1795,TEST_1795,0
1796,TEST_1796,0
1797,TEST_1797,0
1798,TEST_1798,0


In [28]:
submit['label'] = preds
submit

,sample_id,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1795,TEST_1795,8
1796,TEST_1796,0
1797,TEST_1797,1
1798,TEST_1798,1


In [30]:
submit.to_csv("first_trial.csv", index = False)